<a href="https://colab.research.google.com/github/gulabpatel/Computer_Vision/blob/main/Segmentation/SAM_with_Prompts_swimming_pools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mapping swimming pools with text prompts

[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/segment-geospatial/blob/main/docs/examples/swimming_pools.ipynb)
[![image](https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft)](https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/opengeos/segment-geospatial&urlpath=lab/tree/segment-geospatial/docs/examples/swimming_pools.ipynb&branch=main)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/swimming_pools.ipynb)

This notebook shows how to map swimming pools with text prompts and the Segment Anything Model (SAM).

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator.

## Install dependencies

Uncomment and run the following cell to install the required dependencies.

In [ ]:
%pip install segment-geospatial groundingdino-py leafmap localtileserver

In [ ]:
import leafmap
from samgeo import tms_to_geotiff
from samgeo.text_sam import LangSAM

## Create an interactive map

In [ ]:
m = leafmap.Map(center=[34.040984, -118.491668], zoom=19, height="600px")
m.add_basemap("SATELLITE")
m

Map(center=[34.040984, -118.491668], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

## Download a sample image

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [ ]:
bbox = m.user_roi_bounds()
if bbox is None:
    bbox = [-118.4932, 34.0404, -118.4903, 34.0417]

In [ ]:
image = "Image.tif"
tms_to_geotiff(output=image, bbox=bbox, zoom=19, source="Satellite", overwrite=True)

Downloaded image 01/15
Downloaded image 02/15
Downloaded image 03/15
Downloaded image 04/15
Downloaded image 05/15
Downloaded image 06/15
Downloaded image 07/15
Downloaded image 08/15
Downloaded image 09/15
Downloaded image 10/15
Downloaded image 11/15
Downloaded image 12/15
Downloaded image 13/15
Downloaded image 14/15
Downloaded image 15/15
Saving GeoTIFF. Please wait...
Image saved to Image.tif


You can also use your own image. Uncomment and run the following cell to use your own image.

In [ ]:
image = '/content/Image.tif'

Display the downloaded image on the map.

In [ ]:
!pip install localtileserver

In [ ]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

INFO:large_image:Cannot use memcached for caching.
INFO:large_image:Using python for large_image caching


Map(bottom=53597616.0, center=[34.04098613097479, -118.49166316749768], controls=(ZoomControl(options=['positi…

## Initialize LangSAM class

The initialization of the LangSAM class might take a few minutes. The initialization downloads the model weights and sets up the model for inference.

In [ ]:
sam = LangSAM()

final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading: "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth" to /root/.cache/torch/hub/checkpoints/sam_vit_h_4b8939.pth
100%|██████████| 2.39G/2.39G [00:39<00:00, 65.5MB/s]


## Specify text prompts

In [ ]:
text_prompt = "swimming pool"

## Segment the image

Part of the model prediction includes setting appropriate thresholds for object detection and text association with the detected objects. These threshold values range from 0 to 1 and are set while calling the predict method of the LangSAM class.

`box_threshold`: This value is used for object detection in the image. A higher value makes the model more selective, identifying only the most confident object instances, leading to fewer overall detections. A lower value, conversely, makes the model more tolerant, leading to increased detections, including potentially less confident ones.

`text_threshold`: This value is used to associate the detected objects with the provided text prompt. A higher value requires a stronger association between the object and the text prompt, leading to more precise but potentially fewer associations. A lower value allows for looser associations, which could increase the number of associations but also introduce less precise matches.

Remember to test different threshold values on your specific data. The optimal threshold can vary depending on the quality and nature of your images, as well as the specificity of your text prompts. Make sure to choose a balance that suits your requirements, whether that's precision or recall.

In [ ]:
sam.predict(image, text_prompt, box_threshold=0.24, text_threshold=0.24)

## Visualize the results

Show the result with bounding boxes on the map.

In [ ]:
sam.show_anns(
    cmap='Blues',
    box_color='red',
    title='Automatic Segmentation of Swimming Pools',
    blend=True,
)

![](https://i.imgur.com/aXbxCFt.png)

Show the result without bounding boxes on the map.

In [ ]:
sam.show_anns(
    cmap='Blues',
    add_boxes=False,
    alpha=0.5,
    title='Automatic Segmentation of Swimming Pools',
)

![](https://i.imgur.com/9wOkCHC.png)

Show the result as a grayscale image.

In [ ]:
sam.show_anns(
    cmap='Greys_r',
    add_boxes=False,
    alpha=1,
    title='Automatic Segmentation of Swimming Pools',
    blend=False,
    output='pools.tif',
)

![](https://i.imgur.com/lbNtUou.png)

Convert the result to a vector format.

In [ ]:
sam.raster_to_vector("pools.tif", "pools.shp")

Show the results on the interactive map.

In [ ]:
m.add_raster("pools.tif", layer_name="Pools", palette="Blues", opacity=0.5, nodata=0)
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
m.add_vector("pools.shp", layer_name="Vector", style=style)
m

![](https://i.imgur.com/HmFYDAf.png)

## Interactive segmentation

In [ ]:
sam.show_map()

![](https://i.imgur.com/4Q2qpMA.gif)